In [2]:
import pandas as pd
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as sk_tts
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD, Reader

In [2]:
re_data = pd.read_csv('RE_subset.csv')

In [3]:
re_data.head()

,cust_id,acc_id,acc_name
0,454278967,1760062/A,"Air Fly Engine Guard, Black"
1,454278967,1740001/C,"Machined Oil Filler Cap, Silver"
2,454278967,1750083/B,"Military Pannier Pair, Black"
3,454278967,1750049/G,"Pannier Mounting Kit, Drum"
4,454278967,1720034/C,"Machined Front Reservoir Cap, Black, Diagonal"


In [20]:
acc = pd.Series(re_data['acc_name'].unique())

In [28]:
acc = acc.reset_index()
acc.head()

,level_0,index,0
0,0,0,"Air Fly Engine Guard, Black"
1,1,1,"Machined Oil Filler Cap, Silver"
2,2,2,"Military Pannier Pair, Black"
3,3,3,"Pannier Mounting Kit, Drum"
4,4,4,"Machined Front Reservoir Cap, Black, Diagonal"


In [30]:
acc.drop('level_0', axis=1, inplace=True)

In [38]:
acc.rename(columns={'index':'acc_id',0:'acc_name'}, inplace=True)

In [46]:

acc.drop('index', axis=1, inplace=True)

In [48]:
acc['acc_id'] = acc['acc_id']+1
acc.head()

,acc_id,acc_name
0,1,"Air Fly Engine Guard, Black"
1,2,"Machined Oil Filler Cap, Silver"
2,3,"Military Pannier Pair, Black"
3,4,"Pannier Mounting Kit, Drum"
4,5,"Machined Front Reservoir Cap, Black, Diagonal"


In [49]:
re_data.drop('acc_id', axis=1, inplace=True)

In [50]:
re_data.head()

,cust_id,acc_name
0,454278967,"Air Fly Engine Guard, Black"
1,454278967,"Machined Oil Filler Cap, Silver"
2,454278967,"Military Pannier Pair, Black"
3,454278967,"Pannier Mounting Kit, Drum"
4,454278967,"Machined Front Reservoir Cap, Black, Diagonal"


In [13]:
acc_grpby_id.head()

,acc_name
acc_id,
1710008/B,135
1710010/B,212
1720005/A,267
1720021/C,152
1720034/C,79


In [14]:
acc_grpby.shape

(92, 1)

In [52]:
re_data.head()

,cust_id,acc_name
0,454278967,"Air Fly Engine Guard, Black"
1,454278967,"Machined Oil Filler Cap, Silver"
2,454278967,"Military Pannier Pair, Black"
3,454278967,"Pannier Mounting Kit, Drum"
4,454278967,"Machined Front Reservoir Cap, Black, Diagonal"


In [54]:
new_data = pd.merge(re_data,acc, on='acc_name', how='left')

In [55]:
new_data.head()

,cust_id,acc_name,acc_id
0,454278967,"Air Fly Engine Guard, Black",1
1,454278967,"Machined Oil Filler Cap, Silver",2
2,454278967,"Military Pannier Pair, Black",3
3,454278967,"Pannier Mounting Kit, Drum",4
4,454278967,"Machined Front Reservoir Cap, Black, Diagonal",5


In [56]:
new_data['acc_name'].nunique()

92

In [59]:
new_data.shape

(9997, 3)

In [61]:
new_data.to_csv('RE_subset_acc_id.csv', index=False)

## collaborative approach start here

In [3]:
df = pd.read_csv('RE_subset_acc_id.csv')

In [4]:
df.head()

,cust_id,acc_name,acc_id
0,454278967,"Air Fly Engine Guard, Black",1
1,454278967,"Machined Oil Filler Cap, Silver",2
2,454278967,"Military Pannier Pair, Black",3
3,454278967,"Pannier Mounting Kit, Drum",4
4,454278967,"Machined Front Reservoir Cap, Black, Diagonal",5


In [5]:
df['cust_id'].nunique()

3701

In [6]:
df.shape

(9997, 3)

In [11]:
df['rating'] = df['rating'].apply(lambda x : randint(1,5))

In [12]:
df['rating'].value_counts()

1    2045
4    2013
2    1999
5    1970
3    1970
Name: rating, dtype: int64

In [13]:
df.head()

,cust_id,acc_name,acc_id,rating
0,454278967,"Air Fly Engine Guard, Black",1,5
1,454278967,"Machined Oil Filler Cap, Silver",2,1
2,454278967,"Military Pannier Pair, Black",3,1
3,454278967,"Pannier Mounting Kit, Drum",4,2
4,454278967,"Machined Front Reservoir Cap, Black, Diagonal",5,2


In [14]:
df.sample(frac=1)
train, test = sk_tts(df, test_size=0.2, random_state=2)

In [15]:
print(train.shape, test.shape)

(7997, 4) (2000, 4)


In [16]:
train.drop('acc_name', axis=1, inplace=True)

C:\Users\mohittewari\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
train.head()

,cust_id,acc_id,rating
9794,439693528,46,2
3198,613044987,23,2
8589,279511370,18,2
524,906281886,25,5
6402,917941668,44,4


In [18]:
test_user = []
train_users = list(train['cust_id'])
for i in test['cust_id']:
    if i in train_users:
        test_user.append(i)

In [19]:
len(test_user)

1619

In [20]:
test_df = test[test['cust_id'].isin(test_user)]

In [21]:
test_df.shape, test.shape

((1619, 4), (2000, 4))

In [22]:
test_df['cust_id'].nunique()

1159

In [23]:
test_grp = test_df.groupby('cust_id')['acc_id'].apply(list)

In [24]:
test_grp

cust_id
101256620            [50]
101444566    [23, 24, 20]
101862722            [75]
102180638            [79]
103586439            [61]
                 ...     
993581383            [18]
993592875            [44]
994099114            [13]
996854495            [26]
998850334            [26]
Name: acc_id, Length: 1159, dtype: object

In [25]:
type(test_grp)

pandas.core.series.Series

In [26]:
test_grp[101444566]

[23, 24, 20]

In [27]:
train_surprise = Dataset.load_from_df(train, reader=Reader())
trainset = train_surprise.build_full_trainset()

In [28]:
param_grid = {'n_factors':list(range(2,100))}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs.fit(train_surprise)

In [29]:
gs.best_score

{'rmse': 1.437461985767436}

In [30]:
gs.best_params

{'rmse': {'n_factors': 6}}

In [31]:
svd_model = SVD(n_factors=6)
svd_model.fit(trainset)

In [32]:
testset = trainset.build_anti_testset()
predictions = svd_model.test(testset)

In [33]:
predictions[0]

Prediction(uid=439693528, iid=23, r_ui=2.9966237339002126, est=2.8434140063641915, details={'was_impossible': False})

In [38]:
t = pd.DataFrame(predictions)

In [39]:
t.head()

,uid,iid,r_ui,est,details
0,439693528,23,2.996624,2.843414,{'was_impossible': False}
1,439693528,18,2.996624,3.034797,{'was_impossible': False}
2,439693528,25,2.996624,2.762734,{'was_impossible': False}
3,439693528,44,2.996624,2.971402,{'was_impossible': False}
4,439693528,29,2.996624,3.158455,{'was_impossible': False}


In [40]:
test_grp.head()

cust_id
101256620            [50]
101444566    [23, 24, 20]
101862722            [75]
102180638            [79]
103586439            [61]
Name: acc_id, dtype: object

In [53]:
t[t['uid']==101444566].sort_values('est', ascending=False)[:3]

,uid,iid,r_ui,est,details
43709,101444566,89,2.996624,3.279273,{'was_impossible': False}
43732,101444566,90,2.996624,3.203459,{'was_impossible': False}
43692,101444566,68,2.996624,3.169773,{'was_impossible': False}


In [46]:
t[(t['uid']==101444566)]

,uid,iid,r_ui,est,details
43664,101444566,46,2.996624,2.822686,{'was_impossible': False}
43665,101444566,23,2.996624,2.998633,{'was_impossible': False}
43666,101444566,18,2.996624,2.926106,{'was_impossible': False}
43667,101444566,25,2.996624,2.847427,{'was_impossible': False}
43668,101444566,44,2.996624,3.033955,{'was_impossible': False}
...,...,...,...,...,...
43744,101444566,6,2.996624,2.854535,{'was_impossible': False}
43745,101444566,92,2.996624,2.823515,{'was_impossible': False}
43746,101444566,66,2.996624,3.043270,{'was_impossible': False}
43747,101444566,53,2.996624,3.101326,{'was_impossible': False}


In [78]:
pos_count = neg_count = 0
for cust_id,items in test_grp.iteritems():
    top_3_recommendations = set(t[t['uid']==cust_id].sort_values('est',ascending=False)[:3]['iid'])
    items = set(items)
    correct_predictions = top_3_recommendations.intersection(items)
    if len(correct_predictions)>0:
        pos_count += len(correct_predictions)
    else:
        neg_count+=1

In [80]:
print(pos_count, neg_count)

39 1120


In [59]:
#tt=t[t['uid']==101444566].sort_values('est', ascending=False)[:3]['iid']

In [60]:
#tt.head()

43709    89
43732    90
43692    68
Name: iid, dtype: int64

In [70]:
set(tt)

{68, 89, 90}